In [2]:
import mlflow.pytorch
import torch
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn.models import DeepGraphInfomax
from tqdm import tqdm
import pandas as pd
import graph_reinforcement_learning_using_blockchain_data as grl
from graph_reinforcement_learning_using_blockchain_data import config

config.load_dotenv()

2025-05-07 14:55:31.800 | INFO     | graph_reinforcement_learning_using_blockchain_data.config:<module>:12 - PROJ_ROOT path is: /Users/liamtessendorf/Programming/Uni/2_Master/4_FS25_Programming/graph-reinforcement-learning-using-blockchain-data


True

## Load Data

In [3]:
dataset = torch.load(config.FLASHBOTS_Q2_DATA_DIR / "state_graphs.pt", weights_only=False)

In [4]:
max_feats_len = 0
for vals in dataset.values():
    for graphs in vals:
        if graphs.x.shape[1] > max_feats_len:
            max_feats_len = graphs.x.shape[1]

In [5]:
for vals in dataset.values():
    for graphs in vals:
        graphs.x = grl.pad_features(graphs.x, max_feats_len)

In [6]:
data_list = [d for sublist in dataset.values() for d in sublist]

generator = torch.Generator().manual_seed(42)
train_dataset, test_dataset = torch.utils.data.random_split(
    data_list, [0.8, 0.2], generator=generator
)

In [7]:
train_loader = DataLoader(
    train_dataset, batch_size=512, shuffle=True, exclude_keys=["account_mapping"], drop_last=False
)
test_loader = DataLoader(
    test_dataset, batch_size=128, shuffle=False, exclude_keys=["account_mapping"], drop_last=False
)
data_loader = DataLoader(
    data_list, batch_size=128, shuffle=False, exclude_keys=["account_mapping"], drop_last=False
)

## Train DGI

In [7]:
summary = lambda z, *_, batch=None, **__: global_mean_pool(z, batch)


def corruption(data):
    """
    Corrupts the graph data by shuffling node features.
    Accepts a Data/Batch object and returns a corrupted Data/Batch object.
    """
    corrupted_data = data.clone()  # Create a copy to avoid modifying the original data
    num_nodes_total = data.x.size(0)
    # Generate a permutation of node indices within the batch
    perm = torch.randperm(num_nodes_total, device=data.x.device)
    corrupted_data.x = data.x[perm]  # Apply the permutation to shuffle features
    return corrupted_data

In [8]:
num_node_features = max_feats_len
dim_global_features = 0
hidden_channels = 128

device = torch.device("mps")
enc = grl.SAGEEncoder(num_node_features, hidden_channels).to(device)
dgi = DeepGraphInfomax(
    hidden_channels=hidden_channels, encoder=enc, summary=summary, corruption=corruption
).to(device)

In [9]:
enc = grl.pretrain_dgi(train_loader, dgi, device, epochs=20)

2025/05/06 16:06:18 WARNING mlflow.utils.autologging_utils: MLflow pytorch autologging is known to be compatible with 1.9.0 <= torch <= 2.5.1, but the installed version is 2.6.0. If you encounter errors during autologging, try upgrading / downgrading torch to a compatible version, or try upgrading MLflow.
  5%|▌         | 1/20 [00:58<18:32, 58.54s/it]

[DGI] epoch 01  loss=0.6282


 10%|█         | 2/20 [02:12<20:19, 67.77s/it]

[DGI] epoch 02  loss=0.1886


 15%|█▌        | 3/20 [03:48<22:49, 80.55s/it]

...

[DGI] epoch 19  loss=0.0492


100%|██████████| 20/20 [43:22<00:00, 130.13s/it]

[DGI] epoch 20  loss=0.0378



2025/05/06 16:49:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run hilarious-elk-445 at: http://127.0.0.1:8080/#/experiments/330930495026013213/runs/b4b2eb74d0e04aed97df2a607a451fa9
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/330930495026013213


## Train GraphSAGE Classifier using frozen DGI embeddings

In [8]:
model_uri = "mlflow-artifacts:/330930495026013213/b4b2eb74d0e04aed97df2a607a451fa9/artifacts/model"
model = mlflow.pytorch.load_model(model_uri)
enc = model.encoder

In [11]:
for p in enc.parameters():
    p.requires_grad = False

device = torch.device("mps")
model = grl.GraphSAGEClassifier(enc, hidden=128, num_classes=2).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

In [13]:
model, embeddings = grl.run_experiment(
    "DGI GraphSAGE",
    20,
    model,
    train_loader,
    test_loader,
    optimizer,
    criterion,
    device,
    return_embeddings=True,
)

2025/05/07 14:56:59 WARNING mlflow.utils.autologging_utils: MLflow pytorch autologging is known to be compatible with 1.9.0 <= torch <= 2.5.1, but the installed version is 2.6.0. If you encounter errors during autologging, try upgrading / downgrading torch to a compatible version, or try upgrading MLflow.
  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1 starts


  5%|▌         | 1/20 [00:43<13:45, 43.46s/it]

Epoch: 001, Train Loss: 0.0827, Test Acc: 0.9787
Epoch 2 starts


 10%|█         | 2/20 [01:27<13:08, 43.83s/it]

...

Epoch: 019, Train Loss: 0.0401, Test Acc: 0.9874
Epoch 20 starts


100%|██████████| 20/20 [25:54<00:00, 77.72s/it]

Epoch: 020, Train Loss: 0.0397, Test Acc: 0.9878



2025/05/07 15:23:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run capable-bass-603 at: http://127.0.0.1:8080/#/experiments/132032870842317128/runs/de719fc0c8eb45acaba6762076ed4917
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/132032870842317128


## Create Embeddings from pre-trained GNN for downstream tasks

In [ ]:
model_uri = "mlflow-artifacts:/132032870842317128/ba4bedca3761489498f42f57c3986752/artifacts/model"
model = mlflow.pytorch.load_model(model_uri)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
_, _, embeddings = grl.test(model, data_loader, criterion, device, return_embeddings=True)

In [ ]:
emb = {trx_id: emb.cpu().detach().numpy().tolist() for trx_id, emb in embeddings.items()}

In [ ]:
df_embeddings = pd.DataFrame({"transactionHash": emb.keys(), "embeddings": emb.values()})

In [ ]:
df_embeddings.to_csv(
    config.FLASHBOTS_Q2_DATA_DIR / "state_embeddings_pre_trained_128.csv", index=False
)

## Create Embeddings from DGI encoder for downstream tasks

In [ ]:
model_uri = "mlflow-artifacts:/330930495026013213/dc95b44ca9ad4acdbe557b133f6b6653/artifacts/model"
model = mlflow.pytorch.load_model(model_uri)

In [ ]:
model.eval()
unsup_list, y_list = [], []
device = torch.device("mps")
embeddings = {}
with torch.no_grad():
    for data in tqdm(data_loader):
        data = data.to(device)

        z_nodes = model.encoder(data)
        unsup_emb = global_mean_pool(z_nodes, data.batch)

        mapping = {trx_id: emb for trx_id, emb in zip(data.trx_id, unsup_emb)}
        embeddings.update(mapping)

        unsup_list.append(unsup_emb.cpu())
        y_list.append(data.y.cpu())

unsup_X = torch.cat(unsup_list).numpy()
y = torch.cat(y_list).numpy()

In [ ]:
emb = {trx_id: emb.cpu().detach().numpy().tolist() for trx_id, emb in embeddings.items()}
df_embeddings = pd.DataFrame({"transactionHash": emb.keys(), "embeddings": emb.values()})

df_embeddings.to_csv(config.FLASHBOTS_Q2_DATA_DIR / "state_embeddings_dgi_128.csv", index=False)